In [ ]:
#!pip install moviepy

In [1]:
import cv2
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imread
import tensorflow as tf

# ===== import SSD for keras 2 =====
from ssd_v2 import SSD300v2
from ssd_utils import BBoxUtility
# ===== import SSD for keras 2 =====

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'

np.set_printoptions(suppress=True)

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95#0.45
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
voc_classes = ['Aeroplane', 'Bicycle', 'Bird', 'Boat', 'Bottle',
               'Bus', 'Car', 'Cat', 'Chair', 'Cow', 'Diningtable',
               'Dog', 'Horse','Motorbike', 'Person', 'Pottedplant',
               'Sheep', 'Sofa', 'Train', 'Tvmonitor']
NUM_CLASSES = len(voc_classes) + 1

In [3]:
input_shape=(300, 300, 3)
model = SSD300v2(input_shape, num_classes=NUM_CLASSES)
model.load_weights('weights_SSD300.hdf5', by_name=True)
bbox_util = BBoxUtility(NUM_CLASSES)

In [4]:
def get_bbox(xmin, ymin, xmax, ymax):
    return [xmin, ymin, xmax, ymax]

def center_is_near(prev_bbox, bbox):
    IS_NEAR_THRESHOLD = 30
    
    prev_center_x = (prev_bbox[0] + prev_bbox[2])/2.
    prev_center_y = (prev_bbox[1] + prev_bbox[3])/2.
    center_x = (bbox[0] + bbox[2])/2.
    center_y = (bbox[1] + bbox[3])/2.
    
    dist = np.sqrt((prev_center_x - center_x)**2 + (prev_center_y - center_y)**2)
    
    if dist <= IS_NEAR_THRESHOLD:
        return True
    else:
        return False

def draw_boxes(img, preds, results):
    global first_frame_has_car, prev_bboxes, prev_bboxes_len, bbox_disappear_frame_count
    
    # Parse the outputs.
    det_label = results[0][:, 0]
    det_conf = results[0][:, 1]
    det_xmin = results[0][:, 2]
    det_ymin = results[0][:, 3]
    det_xmax = results[0][:, 4]
    det_ymax = results[0][:, 5]

    # Get detections with confidence higher than 0.6.
    top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.6]

    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices].tolist()
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]
    
    bboxes_len = 0
    bboxes = []   
    for i in range(top_conf.shape[0]):
        
        label = int(top_label_indices[i])        
        label_name = voc_classes[label - 1]
        
        if label_name == 'Car':
            bboxes_len += 1
            
            xmin = int(round(top_xmin[i] * img.shape[1]))
            ymin = int(round(top_ymin[i] * img.shape[0]))
            xmax = int(round(top_xmax[i] * img.shape[1]))
            ymax = int(round(top_ymax[i] * img.shape[0]))
            
            if first_frame_has_car or len(prev_bboxes) == 0:
                prev_bboxes.append(get_bbox(xmin, ymin, xmax, ymax))
                first_frame_has_car = False
                prev_bboxes_len = 0
            else:
                has_near_in_prev_bboxes = False
                for i_prev_bbox in range(len(prev_bboxes)):
                    if center_is_near(prev_bboxes[i_prev_bbox], [xmin, ymin, xmax, ymax]):
                        ratiox = 0.5
                        ratioy = 0.65
                        xmin = int((1-ratiox)*xmin + ratiox*prev_bboxes[i_prev_bbox][0])
                        ymin = int((1-ratioy)*ymin + ratioy*prev_bboxes[i_prev_bbox][1])
                        xmax = int((1-ratiox)*xmax + ratiox*prev_bboxes[i_prev_bbox][2])
                        ymax = int((1-ratioy)*ymax + ratioy*prev_bboxes[i_prev_bbox][3])
                        prev_bboxes[i_prev_bbox][0] = xmin 
                        prev_bboxes[i_prev_bbox][1] = ymin
                        prev_bboxes[i_prev_bbox][2] = xmax
                        prev_bboxes[i_prev_bbox][3] = ymax
                        has_near_in_prev_bboxes = True
                        
                if not has_near_in_prev_bboxes:
                    prev_bboxes.append(get_bbox(xmin, ymin, xmax, ymax))
                    
            bboxes.append(get_bbox(xmin, ymin, xmax, ymax))  
           
    if prev_bboxes_len > bboxes_len and bbox_disappear_frame_count < 5:
        for i_prev_bbox in range(len(prev_bboxes)):
            for i_bbox in range(len(bboxes)):
                if not center_is_near(prev_bboxes[i_prev_bbox], bboxes[i_bbox]):
                    cv2.rectangle(img, 
                                  (prev_bboxes[i_prev_bbox][0],prev_bboxes[i_prev_bbox][1]), 
                                  (prev_bboxes[i_prev_bbox][2],prev_bboxes[i_prev_bbox][3]), (0,255,0), 5)
            if len(bboxes) == 0:
                cv2.rectangle(img, 
                              (prev_bboxes[i_prev_bbox][0],prev_bboxes[i_prev_bbox][1]), 
                              (prev_bboxes[i_prev_bbox][2],prev_bboxes[i_prev_bbox][3]), (0,255,0), 5)
        bbox_disappear_frame_count += 1
    else:
        bbox_disappear_frame_count = 0
        prev_bboxes_len = len(bboxes)
        prev_bboxes = bboxes
    for i_bbox in range(len(bboxes)):
        cv2.rectangle(img, (bboxes[i_bbox][0],bboxes[i_bbox][1]), (bboxes[i_bbox][2],bboxes[i_bbox][3]), (0,255,0), 5)  
            
    if len(prev_bboxes) > 10:
        prev_bboxes = []
        bbox_disappear_frame_count = 10
        
    return img


In [5]:
def process_video(input_img):
    
    inputs = []
    #input_img_cropped = input_img[120:720,680:1280,:]
    #img = cv2.resize(input_img_cropped, (300, 300))
    img = cv2.resize(input_img, (300, 300))
    img = image.img_to_array(img)
    inputs.append(img.copy())
    inputs = preprocess_input(np.array(inputs))
    inputs = np.expand_dims(inputs[0], axis=0)
    
    preds = model.predict(inputs, batch_size=1, verbose=0)
    results = bbox_util.detection_out(preds)
    
    final_img = draw_boxes(input_img, preds, results)
    
    return final_img

In [6]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
first_frame_has_car = True
prev_bboxes = []
bbox_disappear_frame_count = 0
prev_bboxes_len = 0

output = 'project_video_video_SSD_smooth_disappear.mp4'
clip1 = VideoFileClip("/tmp/project_video.mp4")
clip = clip1.fl_image(process_video) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video project_video_video_SSD_smooth_disappear.mp4
[MoviePy] Writing video project_video_video_SSD_smooth_disappear.mp4


100%|█████████▉| 1260/1261 [07:17<00:00,  2.87it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_video_SSD_smooth_disappear.mp4 

CPU times: user 45min, sys: 3min 22s, total: 48min 23s
Wall time: 7min 17s


======

In [8]:
!wget -O /tmp/test.jpg https://images.pexels.com/photos/980382/pexels-photo-980382.jpeg


--2018-10-19 23:42:08--  https://images.pexels.com/photos/980382/pexels-photo-980382.jpeg
Resolving images.pexels.com (images.pexels.com)... 104.16.238.112, 104.16.239.112, 104.16.240.112, ...
Connecting to images.pexels.com (images.pexels.com)|104.16.238.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1836911 (1.8M) [image/jpeg]
Saving to: ‘/tmp/test.jpg’

/tmp/test.jpg       100%[===================>]   1.75M  --.-KB/s    in 0.04s   

2018-10-19 23:42:08 (39.9 MB/s) - ‘/tmp/test.jpg’ saved [1836911/1836911]



In [7]:
file_name = '/tmp/test.jpg'
img = image.load_img(file_name, target_size=(300, 300))

In [37]:
inputs = []
#img = cv2.resize(input_img, (300, 300))
img = image.img_to_array(img)
inputs.append(img.copy())
inputs = preprocess_input(np.array(inputs))
inputs = np.expand_dims(inputs[0], axis=0)
preds = model.predict(inputs, batch_size=1, verbose=0)
#results = bbox_util.detection_out(preds)
#final_img = draw_boxes(img, preds, results)

In [38]:
type(preds)

numpy.ndarray

In [39]:
type(inputs)

numpy.ndarray

In [41]:
import json
payload = json.dumps(inputs.tolist())
type(payload)

str

__Simulate what happens on the endpoint__

In [43]:
parsed = json.loads(payload)
type(parsed)

list

In [44]:
data = np.array(parsed)
type(data)

numpy.ndarray

In [47]:
predictions = model.predict(data, batch_size=1, verbose=0)
type(predictions)

numpy.ndarray

In [49]:
result = json.dumps(predictions.tolist())
type(result)

str

__simulate whats happens on the client__

In [51]:
response = np.array(json.loads(result))

In [52]:
results = bbox_util.detection_out(response)

In [53]:
final_image = draw_boxes(img, response, results)

NameError: global name 'prev_bboxes_len' is not defined